# It's a Machine and Natural: Generation

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_gum
from collections import defaultdict

seed = 691
all_docs = load_gum(num_articles = 0, seed = seed)
nsamp = min(125, len(all_docs)-1) ## this is the size of the test document base..., so nsamp=125 => test == 125 & train == 25!
test_docs = all_docs[:nsamp]
train_docs = all_docs[nsamp:]
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
covering_vocab = set([t for d in covering for s in d for t in s])

layers = [[[[row[2] for row in s] for s in d['conllu']] for d in train_docs],
          [[[row[3] for row in s] for s in d['conllu']] for d in train_docs]]
ltypes = ['lem','pos']

model = LM(form2ind={"<sos>": 0, "<pad>": 1}, covering_vocab = covering_vocab)
model.init(m = 10, zeta = 0.01, positional = True, seed = seed)
model.fit(docs, f'GUM-{nsamp}', covering = covering, layers = layers, ltypes = ltypes)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 158109.89it/s]


Training tokenizer...


Fitting:   8%|█████████▉                                                                                                                  | 8/100 [00:04<00:48,  1.88it/s]


Built a vocabulary of 3597 types
Tokenizing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:20<00:00,  1.21it/s]


...

Absorbing pos-layer co-occurrences...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00,  8.65it/s]


Computing marginal statistics...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1549283/1549283 [00:05<00:00, 262845.75it/s]


Done.
Model params, tokens, contexts, and % capacity used: 1549283 6977 139539 0.159


In [2]:
model.generate()

0it [00:00, ?it/s]


The

In [3]:
model.generate(seed = seed)

0it [00:00, ?it/s]


The

In [4]:
model.generate(seed = seed)

0it [00:00, ?it/s]


The

In [5]:
model.generate(m = 25, generate_next = 's', seed = seed)

0it [00:00, ?it/s]


TheGetIslandteaspoonis not likely to help.very

In [6]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.25)

0it [00:00, ?it/s]


WhenWhenthat that can give a Basic line with chicken something on as

In [7]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99)

0it [00:00, ?it/s]


WhenWhenthat that can give a Basic line with chicken something on as

In [8]:
# model.generate(m = 25, generate_next = 's', seed = 691, prose = 0.75, focus = 0.25) # focus is good, but lemmas are expensive to use

In [9]:
# model.generate(m = 25, generate_next = 's', seed = 691, prose = 0.99, focus = 0.75, jargon = 0.25)

In [10]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001)

0it [00:00, ?it/s]


WhenWhenthat that can give a Basic line with chicken something on as

In [11]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.01)

0it [00:00, ?it/s]


Howuntilararwill too simple simple or by something from it bit e

In [12]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.1)

0it [00:00, ?it/s]


with You do t n’t creat north-south salt like The pockets will known

In [13]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.99)

0it [00:00, ?it/s]


TraditionallyLakesgenerallyexaggeratedwinter thing tubs pridefulcuriouscommercialignorantmisogynistcourseexcuseaccustomedleotardStirpossessdrawWriteStretsurvival

In [14]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.75, noise = 0.99)

0it [00:00, ?it/s]


TraditionallyLakesgenerallyexaggeratedschooleAlso secondsFollowfacilitateirritatedwindseasonangertraditionalAlternativelyChancesdoubleusernamerivalriesdirectionreligiousreligiousvisually

In [15]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.9999)

0it [00:00, ?it/s]


annuallyAfrican[Even another ad commonly 

In [16]:
model.generate(m = 25, generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.75, noise = 0.9999)

0it [00:00, ?it/s]


respectfullyAfrican[Even another ad commonly 

In [17]:
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001)

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9845.78it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 73.90it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13273.11it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.61s/it]


on a air baking mixture from 5 milk minutes seconds Creek American Viking

In [18]:
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.01)

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9776.93it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 70.71it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11428.62it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.65s/it]


from a story from amount mix mile system wai flavor as The mark

In [19]:
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.99)

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5461.33it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 73.39it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13706.88it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.70s/it]


from a Part at not hotels question likely n of size mix like

In [20]:
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.9999)

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9554.22it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.86it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9020.01it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.61s/it]


from a stress at rid cups by list plain option bottom From a

In [21]:
from collections import Counter
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.9999,
               Td = Counter([t for doc in docs[-2][0:2] for t in model.tokenizer.tokenize(doc)]))

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5777.28it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 88.76it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12520.31it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.61s/it]


from a serve into bludgers mirror an containers up t Roman Steps garden

In [22]:
from collections import Counter
print(docs[-2][2][:60])
model.generate(m = 25, docs = [[docs[-2][2][:60]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.25,
               Td = Counter([t for doc in docs[-2][0:2] for t in model.tokenizer.tokenize(doc)]))

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8338.58it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 92.98it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8192.00it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.63s/it]


from a hotels from basis back mice by some population than than mile

In [23]:
from collections import Counter
print(docs[-2][2][:60])
model.generate(m = 75, docs = [[docs[-2][2][:60]]], 
               generate_next = 'd', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.25,
               Td = Counter([t for doc in docs[-2][0:2] for t in model.tokenizer.tokenize(doc)]))

The Incas treated the crop as sacred and referred to quinoa 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9157.87it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 88.90it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9078.58it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.65s/it]


from a hotels from basis back mice by some population than than mile around an can soy wrap end as The mark as their flour or world from summer bottom n an players garden cook wai For everything

In [24]:
print(tdocs[1][4][:39])  
model.generate(m = 25, docs = [[tdocs[1][4][:39]]], 
               generate_next = 's', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.25,
               Td = Counter([t for tdoc in tdocs[1][:4] for t in model.tokenizer.tokenize(tdoc)]))

Additionally, diversity has been shown 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10979.85it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 96.02it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15592.21it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.10s/it]


t very dangerous out s words Great from all goal Trail Mukalla both

In [25]:
print(tdocs[1][4][:39])  
model.generate(m = 75, docs = [[tdocs[1][4][:39]]], 
               generate_next = 'd', seed = seed, prose = 0.99, focus = 0.0, jargon = 0.25, noise = 0.0001, resonance = 0.25,
               Td = Counter([t for tdoc in tdocs[1][:4] for t in model.tokenizer.tokenize(tdoc)]))

Additionally, diversity has been shown 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8224.13it/s]


Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 94.35it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 6384.02it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.07s/it]


t very dangerous out s words Great from all goal Trail Mukalla both not question wrap number do system arms t want not continue t s northe slight life than at summer earlier up mix ch serve ing